# Effectiveness Recalculator (for post-test synthesis)

In [1]:
import pandas as pd
import numpy as np
import glob, os
import json
from pandas.io.parsers import read_csv

In [2]:
combined_csv = read_csv("test_synthesizer_results.csv")
combined_csv = combined_csv.drop(['TP', 'FP', 'FN', 'num_TP', 'num_FN', 'num_FP', 'accuracy_precision', 'accuracy_recall', 'effectiveness_precision', 'effectiveness_recall'], axis=1)
combined_csv['transferred'] = combined_csv['transferred'].apply(json.loads)
combined_csv

,test_id,method,transferred,source,target,gui_mapper,correct,incorrect,missed,nonExist,num_correct,num_incorrect,num_missed,num_nonExist
0,0,<Ebay.RepresentativeTests: void testSignIn()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/sign_in_button', 'id@com.g...",[],[],['NONE'],4,0,0,1
1,1,<Ebay.RepresentativeTests: void testSignUp()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,['xpath@/hierarchy/android.widget.FrameLayout/...,['id@com.groupon:id/fragment_log_in_sign_up_em...,[],"['NONE', 'NONE']",2,3,0,2
2,2,<Ebay.RepresentativeTests: void testAbout()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,['xpath@/hierarchy/android.widget.FrameLayout/...,['id@com.groupon:id/my_stuff'],[],['NONE'],1,1,0,1
3,3,<Ebay.RepresentativeTests: void testAccount()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/sign_in_button', 'id@com.g...",[],[],"['NONE', 'NONE']",5,0,0,2
4,4,<Ebay.RepresentativeTests: void testCategory()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/category_icon', 'xpath@/hi...",[],[],[],2,0,0,0
5,5,<Ebay.RepresentativeTests: void testHelp()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,[],[],[],"['NONE', 'NONE']",0,0,0,2
6,6,<Ebay.RepresentativeTests: void testMenu()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,[],[],[],['NONE'],0,0,0,1
7,7,<Ebay.RepresentativeTests: void testSearch()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],2,0,0,0
8,8,<Ebay.RepresentativeTests: void testFilter()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],4,0,0,0
9,9,<Ebay.RepresentativeTests: void testDetail()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],3,0,0,0


In [3]:
combined_csv.iloc[0]['transferred']

[{'input': None,
  'id_or_xpath': 'id@com.groupon:id/sign_in_button',
  'action': 'click',
  'case': 'correct'},
 {'input': None, 'id_or_xpath': 'NONE', 'action': 'click', 'case': 'nonExist'},
 {'input': '***REMOVED***',
  'id_or_xpath': 'id@com.groupon:id/fragment_log_in_sign_up_email',
  'action': 'sendKeys',
  'case': 'correct'},
 {'input': '***REMOVED***!',
  'id_or_xpath': 'id@com.groupon:id/fragment_log_in_sign_up_password',
  'action': 'sendKeys',
  'case': 'correct'},
 None,
 {'input': None,
  'id_or_xpath': 'id@com.groupon:id/sign_in_button',
  'action': 'click',
  'case': 'correct'}]

In [4]:
def remove_none(test):
    return list(filter(None.__ne__, test))

In [5]:
ground_truth_tests = [read_csv(path, names=["method", "json"]) for path in glob.glob("../src/test_csv/*.csv")]
ground_truth_tests = pd.concat(ground_truth_tests)
ground_truth_tests['json'] = ground_truth_tests['json'].apply(json.loads)

In [6]:
def evaluate_effectiveness(test):
    transfer_events = set()
    for gui_event in test['transferred']:
        #print(gui_event)
        if 'id_or_xpath' in gui_event:
            transfer_events.add(gui_event['id_or_xpath'])
        elif gui_event['id'] == "":
            transfer_events.add(gui_event['xpath'])
        else:
            transfer_events.add(gui_event['id'])
    
    target_method = test['method'].replace(test['source'], test['target'])
    target_test = ground_truth_tests.loc[ground_truth_tests['method'] == target_method]
    if target_test.shape[0] == 0:
        target_events = set()
    else:
        target_events = {gui_event['id_or_xpath'] for gui_event in target_test.iloc[0]['json']}

    cases= {}
    cases['TP'] = transfer_events & target_events
    cases['FP'] = transfer_events - target_events
    cases['FN'] = target_events - transfer_events
                
    return cases

In [7]:
def count_cases(test):
    cases = {}
    cases['num_TP'] = len(test['TP'])
    cases['num_FP'] = len(test['FP'])
    cases['num_FN'] = len(test['FN'])
    return cases

In [8]:
def calc_precision_recall(test):
    fractions = {}
    try:
        fractions['effectiveness_precision'] = test['num_TP'] / (test['num_TP'] + test['num_FP'])
    except ZeroDivisionError:
        fractions['effectiveness_precision'] = np.NaN
    try:
        fractions['effectiveness_recall'] = test['num_TP'] / (test['num_TP'] + test['num_FN'])
    except ZeroDivisionError:
        fractions['effectiveness_recall'] = np.NaN
    return fractions

In [9]:
combined_csv['transferred'] = combined_csv['transferred'].apply(remove_none)
combined_csv = pd.concat([combined_csv, combined_csv.apply(evaluate_effectiveness, axis=1).apply(pd.Series)], axis=1)
combined_csv = pd.concat([combined_csv, combined_csv.apply(count_cases, axis=1).apply(pd.Series)], axis=1)
combined_csv = pd.concat([combined_csv, combined_csv.apply(calc_precision_recall, axis=1).apply(pd.Series)], axis=1)
combined_csv

,test_id,method,transferred,source,target,gui_mapper,correct,incorrect,missed,nonExist,...,num_missed,num_nonExist,TP,FP,FN,num_TP,num_FP,num_FN,effectiveness_precision,effectiveness_recall
0,0,<Ebay.RepresentativeTests: void testSignIn()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/sign_in_button', 'id@com.g...",[],[],['NONE'],...,0,1,{id@com.groupon:id/fragment_log_in_sign_up_ema...,{NONE},"{id@com.groupon:id/my_stuff, id@com.groupon:id...",3,1,2,0.750000,0.600000
1,1,<Ebay.RepresentativeTests: void testSignUp()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,['xpath@/hierarchy/android.widget.FrameLayout/...,['id@com.groupon:id/fragment_log_in_sign_up_em...,[],"['NONE', 'NONE']",...,0,2,{id@com.groupon:id/fragment_log_in_sign_up_ema...,{NONE},"{id@com.groupon:id/my_stuff, id@com.groupon:id...",5,1,2,0.833333,0.714286
2,2,<Ebay.RepresentativeTests: void testAbout()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,['xpath@/hierarchy/android.widget.FrameLayout/...,['id@com.groupon:id/my_stuff'],[],['NONE'],...,0,1,"{id@com.groupon:id/my_stuff, xpath@/hierarchy/...",{NONE},{},2,1,0,0.666667,1.000000
3,3,<Ebay.RepresentativeTests: void testAccount()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/sign_in_button', 'id@com.g...",[],[],"['NONE', 'NONE']",...,0,2,{id@com.groupon:id/my_stuff},"{NONE, id@com.groupon:id/fragment_log_in_sign_...",{},1,4,0,0.200000,1.000000
4,4,<Ebay.RepresentativeTests: void testCategory()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,"['id@com.groupon:id/category_icon', 'xpath@/hi...",[],[],[],...,0,0,"{id@com.groupon:id/category_icon, xpath@/hiera...",{},{},2,0,0,1.000000,1.000000
5,5,<Ebay.RepresentativeTests: void testHelp()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,[],[],[],"['NONE', 'NONE']",...,0,2,{},{NONE},{},0,1,0,0.000000,NaN
6,6,<Ebay.RepresentativeTests: void testMenu()>,"[{'input': None, 'id_or_xpath': 'NONE', 'actio...",Ebay,Groupon,Perfect,[],[],[],['NONE'],...,0,1,{},{NONE},{},0,1,0,0.000000,NaN
7,7,<Ebay.RepresentativeTests: void testSearch()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],...,0,0,"{id@com.groupon:id/global_search_button_text, ...",{},{},2,0,0,1.000000,1.000000
8,8,<Ebay.RepresentativeTests: void testFilter()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],...,0,0,"{id@com.groupon:id/global_search_button_text, ...",{},{id@com.groupon:id/done_button},4,0,1,1.000000,0.800000
9,9,<Ebay.RepresentativeTests: void testDetail()>,"[{'input': None, 'id_or_xpath': 'id@com.groupo...",Ebay,Groupon,Perfect,['id@com.groupon:id/global_search_button_text'...,[],[],[],...,0,0,"{id@com.groupon:id/global_search_button_text, ...",{},{},3,0,0,1.000000,1.000000
